# Modelado semántico
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

El modelado léxico tiene importantes usos en la recuperación de información; los sistemas pueden ser altamente escalables si hay una implementación adecuada en índice invertido. Sin embargo, sufren de diversos problemas debido a su naturaleza:

- Las consultas deben ser expresadas con los mismos términos que los documentos.
- Los términos con múltiples significados pueden introducir falsos positivos en los resultados.
- En general se tienen problemas con la sinónimia, la homonimia, y la hiponimia, así como las variaciones léxicas resultantes de errores ortográficos, ya sean intencionados o involuntarios.

Aunque existen soluciones dos grandes vertientes de modelado semántico:
- Modelado del vocabulario: Intententa asignar cada palabra del vocabulario a un _concepto_ o significado _semántico_, que esta representado por un vector de alta dimensión (embedding).
- Modelado del lenguaje: Intenta modelar el lenguaje, normalmente aprendiendo que palabras pueden seguir a un contexto dado. La diferencia principal al modelado de vocabulario es que cuando se modela el lenguaje se tiene un modelo sensible al contexto, y por lo tanto, para una misma palabra, el contexto completo derminará su vector _embedding_. 

Aunque existen diversos intentos de modelado semántico en la literatura clásica, hasta tiempos recientes es cuando ha tomado gran interés debido a los impresionantes avances en redes neuronales profundas que han permitido aprender estos modelos de enormes colecciones, capturando una semántica profunda. Los grandes modelos de lenguaje han abierto nuevas posibilidades sobre las aplicaciones posibles en recuperación de información.

## Sobre las representaciones de documentos
Los modelos semánticos de palabras, anteriormente discutidos, son insuficientes para la representación de documentos; para conseguir representaciones de documentos hay diferentes estrategias. Por ejemplo, es posible generar combinaciones lineales de los embeddings individuales para generar vector embedding de documento. Algunos modelos basados en redes neuronales, generan embeddings de documento por diseño.


## Organización del resto de la unidad
Esta únidad esta dividida en dos notebooks. El primero (este documento) está dedicado al modelado semántico de vocabularios, mientras que el segundo se enfoca a modelos de lenguaje que trabajen sobre documentos. 

# Modelado semántico de vocabularios
El modelado basado en vocabularios o léxico tiene una gran cantidad de aplicaciones, las cuales son suelen explotar la gran cantidad de grados de libertad en el preprocesamiento, toquenizado y pesado, para ajustar a las tareas deseadas. Sin embargo, dicho modelado tiene problemas basados en la semántica:

- si las consultas tienen palabras no conocidas (fuera de vocabulario), no es posible ligar a palabras similares por semántica 
- las palabras similares en el vocabulario pero no iguales, no puede relacionarse para reducir un vocabulario, o para cualquier otro efecto
- las palabras que se escriben igual pero que tienen significado idéntico serán puestas en la misma bolsa

De manera más detallada, se tendrán problemas al manejar sinónimos, hipónimos, antónimos, etc. Adicionalmente, los errores serán tratados como palabras diferentes y no ajustados al mejor calce conocido (a menos que exista un corrector ortográfico entre el preprocesamiento).

Estos problemas se buscan reducir con representaciones semánticas de los vocabularios y de los textos, esto sería equivalente. El intento sería buscar por conceptos más que por palabras que lo describan.
En este punto aparecen los modelos semánticos, los cuales suelen ser representaciones vectoriales densas _embeddings_, de alta dimensión pero mucho menor que las representaciones léxicas (i.e., pasarían a unos pocos cientos en lugar de varias decenas de miles o cientos de miles de coordinadas).

# Word embeddings
Las representaciones de vectores densos de las palabras se suelen construir utilizando la [_hipótesis distribucional_](https://aclweb.org/aclwiki/Distributional_Hypothesis), que se puede resumir que las palabras con contextos similares (palabras que la rodean) tienen un significado similar. Entonces, para obtener la semántica, es necesario comparar contextos; lo anterior se puede llevar a cabo de diferentes maneras.

- Latent semantic analysis (LSA): La matriz $W_{m,n}^*$  con pesos basados en frecuencias es usada para reducir el numero de filas (palabras) mediante _Singular Value Decomposition_ (SVD). La similitud adecuada entre vectores en el espacio de dimensión reducida es el coseno. Ver [@Dumais2004] para más información.
- Latent semantic indexing (LSI): Similar a LSA pero usando esquemas de pesado diferentes, adicionalmente se usa Trucated SVD (TSVD) en lugar de SVD ya que puede seleccionar componentes más importantes. Para más detalles ver [@Hofman1999].
- Word2Vec. Se construye una red neuronal con una matriz de pesos del tamaño del vocabulario por una dimensión seleccionada, a partir de los contextos. Revisa los textos de entrenamiento mediante una ventana móvil. De manera especial, se tiene que el centro de la ventana es la palabra objetivo y el contexto esta a su alrededor. Se intenta predecir mediante el contexto la palabra central, y se ajusta mediante propagación hacia atrás. La función de perdida suele ser [softmax](https://en.wikipedia.org/wiki/Softmax_function) o hierachical softmax. Los embeddings no son la capa de salida sino una capa atrás. Para más detalles ver [@MCCD2013].
- Glove. Se utiliza una matriz de coocurrencia de términos dentro de una ventana deslizante que analiza una colección de textos. La matriz es altamente dispersa y de alta dimensión. Plantea un problema de optimización lineal para capturar la información más útil y bajar su dimensión. Los detalles se pueden consultar en [@PSM2014]. 
- FastText. Muy similar a Word2Vec pero añade manejo de palabras fuera de diccionario mediante subwords (similar a q-gramas de carácteres por cada palabra). Ver [@BGJM2017] para mayores detalles.

## Nota
Este notebook esta basado en <https://github.com/sadit/SimilaritySearchDemos/blob/main/Glove/Glove.ipynb>. En el notebook original se muestran visualizaciones basadas en todos los vecinos cercanos (se verán más adelante en el curso).

# Ejemplo

El resto del documento muestra como usar word embeddings Glove para precalculados, para resolver tareas de vecinos cercanos y resolución de analogías.

In [1]:
using Pkg
Pkg.activate(".")
using SimilaritySearch, Plots, LinearAlgebra, Embeddings, HypertextLiteral

  Activating project at `~/Cursos/IR-2024/Unidades`


In [2]:
function load_dataset()
    emb = load_embeddings(GloVe, 2)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end

    db = MatrixDatabase(emb.embeddings)
    db, emb.vocab
end

function create_index()
    db, vocab = load_dataset()
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index)
    optimize!(index, MinRecall(0.9))
    index, vocab
end


create_index (generic function with 1 method)

### El índice métrico se crea, además se toma el vocabulario

In [3]:
@time index, vocab = create_index()
vocab2id = Dict(w => i for (i, w) in enumerate(vocab));

 36.649847 seconds (7.40 M allocations: 3.125 GiB, 3.43% gc time, 5.22% compilation time: 3% of which was recompilation)


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

In [4]:
let k = 32
    @time iI, _ = allknn(index, k)
    # WARNING: Don't run the following line, it takes too much time
    @time gI, _ = allknn(ExhaustiveSearch(; db=index.db, dist=index.dist), k)
    macrorecall(gI, iI)
end

 28.015977 seconds (2.28 M allocations: 5.148 GiB, 0.90% gc time, 2.45% compilation time)
360.393454 seconds (230.26 k allocations: 112.083 MiB, 0.07% compilation time)


0.937028671875

### Búsqueda y presentación de los resultados

In [5]:
function search_and_display(index, vocab, q, res, k, qword)
    res = reuse!(res, k)
    @time search(index, q, res)

    L = []
    for (j, p) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$(vocab[p.id])</td><td>$p.id</td><td>$(round(p.weight, digits=3))</td></tr>")
    end

    display(@htl """<h2>resultados for "$qword"</h2>
    <table>
    <th>  <td>word</td> <td>id</td> <td>dist</td> </th>
        $L
    </table>
    """)
end

function search_and_display(index, vocab, qid::Integer, res, k=maxlength(res))
    search_and_display(index, vocab, index[qid], res, k, vocab[qid])
end

display(@htl "<h1>Ejemplos de búsqueda (aleatorios)</h1>")
res = KnnResult(7)
for i in 1:3
    for qid in rand(1:length(vocab))
        search_and_display(index, vocab, qid, res)
    end
end

<h1>Ejemplos de búsqueda (aleatorios)</h1>

  0.000414 seconds (3 allocations: 1.625 KiB)


1,oxalic,"IdWeight(0x00020564, 0.0f0).id",0.0
2,tartaric,"IdWeight(0x00028f11, 0.23522043f0).id",0.235
3,benzoic,"IdWeight(0x00024415, 0.24037927f0).id",0.24
4,picric,"IdWeight(0x00039f8a, 0.25700676f0).id",0.257
5,salicylic,"IdWeight(0x0001ff29, 0.2570488f0).id",0.257
6,ferulic,"IdWeight(0x00049f23, 0.27578282f0).id",0.276
7,hypochlorous,"IdWeight(0x0004704b, 0.27816248f0).id",0.278


  0.000336 seconds (2 allocations: 32 bytes)


1,financieras,"IdWeight(0x0004f268, 5.9604645f-8).id",0.0
2,culturales,"IdWeight(0x0004a96a, 0.17610365f0).id",0.176
3,ambientales,"IdWeight(0x0005b339, 0.18060797f0).id",0.181
4,entidades,"IdWeight(0x0005f250, 0.20086998f0).id",0.201
5,camaras,"IdWeight(0x000482f0, 0.20159829f0).id",0.202
6,instituciones,"IdWeight(0x0002311f, 0.20482886f0).id",0.205
7,franquicias,"IdWeight(0x00056f07, 0.2072379f0).id",0.207


  0.001120 seconds (3 allocations: 5.906 KiB)


1,placente,"IdWeight(0x00023cc7, 5.9604645f-8).id",0.0
2,dorta,"IdWeight(0x0005d8e2, 0.36412215f0).id",0.364
3,klimowicz,"IdWeight(0x000226f5, 0.36626738f0).id",0.366
4,cagna,"IdWeight(0x00031e4f, 0.39715785f0).id",0.397
5,simeone,"IdWeight(0x0000d3e7, 0.4072147f0).id",0.407
6,chamot,"IdWeight(0x0001fe57, 0.43776006f0).id",0.438
7,rivarola,"IdWeight(0x000265fd, 0.44890302f0).id",0.449


In [6]:
search_and_display(index, vocab, vocab2id["glove"], res, 15)

  0.000691 seconds (4 allocations: 7.500 KiB)


1,glove,"IdWeight(0x00002aab, 0.0f0).id",0.0
2,ball,"IdWeight(0x0000043c, 0.36888665f0).id",0.369
3,gloves,"IdWeight(0x00002a38, 0.36894822f0).id",0.369
4,plate,"IdWeight(0x0000110d, 0.3791324f0).id",0.379
5,pads,"IdWeight(0x00003e6a, 0.4182728f0).id",0.418
6,infield,"IdWeight(0x0000415b, 0.43464988f0).id",0.435
7,bounced,"IdWeight(0x000023c2, 0.43528706f0).id",0.435
8,hat,"IdWeight(0x000015fa, 0.43809587f0).id",0.438
9,bat,"IdWeight(0x0000133e, 0.43897963f0).id",0.439
10,fielder,"IdWeight(0x00002801, 0.44676453f0).id",0.447
11,pitch,"IdWeight(0x00000c1b, 0.45033383f0).id",0.45


In [7]:
# Analogias

In [8]:
function analogy(a, b, c, k)
	v = index[vocab2id[a]] - index[vocab2id[b]] + index[vocab2id[c]]
	normalize!(v)
	search_and_display(index, vocab, v, res, k, "<$a> - <$b> + <$c>")
end

analogy("father", "man", "woman", 5)
analogy("fireman", "man", "woman", 5)
analogy("policeman", "man", "woman", 5)
analogy("mississippi", "usa", "france", 5)

  0.000100 seconds (3 allocations: 1.625 KiB)


1,mother,"IdWeight(0x00000329, 0.097538054f0).id",0.098
2,daughter,"IdWeight(0x0000046c, 0.13246548f0).id",0.132
3,wife,"IdWeight(0x000002bf, 0.14651567f0).id",0.147
4,father,"IdWeight(0x00000276, 0.14774317f0).id",0.148
5,husband,"IdWeight(0x00000530, 0.17211103f0).id",0.172


  0.000184 seconds (2 allocations: 32 bytes)


1,fireman,"IdWeight(0x00006ad1, 0.15624285f0).id",0.156
2,firefighter,"IdWeight(0x00003dc4, 0.30260533f0).id",0.303
3,paramedic,"IdWeight(0x00008431, 0.39318788f0).id",0.393
4,rescuer,"IdWeight(0x0000af73, 0.4389633f0).id",0.439
5,passerby,"IdWeight(0x0000d210, 0.45855498f0).id",0.459


  0.000090 seconds (2 allocations: 32 bytes)


1,policeman,"IdWeight(0x00001ac9, 0.14374709f0).id",0.144
2,wounding,"IdWeight(0x000017e6, 0.2848916f0).id",0.285
3,policemen,"IdWeight(0x00001378, 0.29457688f0).id",0.295
4,passerby,"IdWeight(0x0000d210, 0.3055392f0).id",0.306
5,wounded,"IdWeight(0x00000570, 0.33116198f0).id",0.331


  0.000541 seconds (3 allocations: 5.906 KiB)


1,louisiana,"IdWeight(0x00000f04, 0.4302075f0).id",0.43
2,france,"IdWeight(0x00000184, 0.46778142f0).id",0.468
3,rhine,"IdWeight(0x00003685, 0.48834217f0).id",0.488
4,coast,"IdWeight(0x000003bb, 0.50627756f0).id",0.506
5,brittany,"IdWeight(0x00003e05, 0.50635386f0).id",0.506


# Actividades 
- Como se menciono, el uso de redes neuronales se vuelve muy popular para aprender la semántica desde el texto. Comente las razones justificando sus comentarios en términos de aprendizaje y requerimientos computacionales.
- Discuta que tipo de métrica se usa y porque.
- ¿Cómo se podrían usar los word-embeddings para búsqueda de documentos completos?
- Como ajustaría la métrica.
- Si usa Julia, revisé el paquete `Embeddings.jl`, `Word2Vec.jl`.
- Si usa Python, revisé los paquetes `fastText` y `word2vec`.
- Glove <https://nlp.stanford.edu/projects/glove/>
- Reproduzca el ejercicio de este notebook (no reproduzca la búsqueda exhaustiva, ya que le tomará muchísimo tiempo), use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Anote sus reflexiones.

# Bibliografía
- [Dumais2004] Dumais, S. T. (2004). Latent semantic analysis. Annu. Rev. Inf. Sci. Technol., 38(1), 188-230.
- [Hofmann1999] Hofmann, T. (1999, August). Probabilistic latent semantic indexing. In Proceedings of the 22nd annual international ACM SIGIR conference on Research and development in information retrieval (pp. 50-57).
- [MCCD2013]: Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781.
- [PSM2014] Pennington, J., Socher, R., & Manning, C. D. (2014, October). Glove: Global vectors for word representation. In Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP) (pp. 1532-1543).
- [BGJM2017]: Bojanowski, P., Grave, E., Joulin, A., & Mikolov, T. (2017). Enriching word vectors with subword information. Transactions of the association for computational linguistics, 5, 135-146.
- [VSPU2017] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. Advances in neural information processing systems, 30.